<a href="https://colab.research.google.com/github/123456789jjacg/Repositorio-Trabajos/blob/main/reporte_de_metas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reporte de cumpimientos de metas FACOGEM

## Seleccion de datos

In [ ]:
import pandas as pd

gsheetid="19RBmWjQ3u8ge7GPoD1NzISSGAzAvEu_CcbS1zawm4qA" #Reporte OC

sheetid="0"
sheetid1="1247807390"
sheetid2="1942764991"
sheetid3="1081813947"


url=f"https://docs.google.com/spreadsheets/d/{gsheetid}/export?format=csv&gid={sheetid}&format" #Pedidos pendiente SAP
url1=f"https://docs.google.com/spreadsheets/d/{gsheetid}/export?format=csv&gid={sheetid1}&format" #Articulo
url2=f"https://docs.google.com/spreadsheets/d/{gsheetid}/export?format=csv&gid={sheetid2}&format" #Familia
url3=f"https://docs.google.com/spreadsheets/d/{gsheetid}/export?format=csv&gid={sheetid3}&format" #Meta

In [ ]:
pedidos=pd.read_csv(url)
pedidos["Fecha de contabilización"]=pedidos["Fecha de contabilización"].astype("datetime64[ns]")
Año=pedidos["Fecha de contabilización"].dt.strftime("%Y")
pedidos['trimestre'] = pedidos['Fecha de contabilización'].dt.quarter #Aqui allamos el trimenes
pedidos["Año"]=Año
pedidos.rename(columns={"Número de documento":"OC","Fecha de contabilización":"Fecha","Nombre de cliente/proveedor":"Proveedor","Número de artículo":"Codigo SAP"},inplace=True)
pedidos=pedidos.iloc[:,[0,2,3,5,7,8,9,10,11]]


pedidos["Año"]=pedidos["Año"].str.replace(",","")
pedidos["Año"]=pedidos["Año"].astype("int64")
pedidos=pedidos[(pedidos["Proveedor"]=="SCHNEIDER ELECTRIC PERU S.A.") & (pedidos["Año"]>2022)]
pedidos["Total bruto"]=pedidos["Total bruto"].str.replace(",","")
pedidos["Total bruto"]=pedidos["Total bruto"].astype("float64")
pedidos["Cantidad"]=pedidos["Cantidad"].astype("float64")

Art=pd.read_csv(url1)
Art=Art.iloc[:,[0,1,3]]
Art. rename(columns={"Id Adicional":"Cod Schneider","Descripcion de articulo":"Descripcion","Número de artículo":"Codigo SAP"},inplace=True)
Art.drop_duplicates(subset="Cod Schneider",inplace=True)
Art.dropna(subset="Cod Schneider",inplace=True)

familia=pd.read_csv(url2)
familia=familia.iloc[:,[0,2,3]]
familia.rename(columns={"Referencia":"Cod Schneider","Division":"Familia","Actividad":"Sub familia"},inplace=True)

pedidos=pedidos.merge(Art,left_on="Codigo SAP",right_on="Codigo SAP",validate="many_to_one",how="left")
pedidos=pedidos.merge(familia,left_on="Cod Schneider",right_on="Cod Schneider",validate="many_to_one",how="left")

pedidos.rename(columns={"Cantidad":"Cantidad total"},inplace=True)
pedidos["Cantidad atentida"]=pedidos["Cantidad total"]-pedidos["Cantidad abierta restante"]
pedidos["Monto total atendido"]=(pedidos["Total bruto"]/pedidos["Cantidad total"])*pedidos["Cantidad atentida"]
pedidos["Monto total no atendido"]=(pedidos["Total bruto"]/pedidos["Cantidad total"])*pedidos["Cantidad abierta restante"]


tri={1:"Trimestres 1",2:"Trimestre 2", 3:"Trimestre 3",4:"Trimestre 4"}
pedidos["trimestre"]=pedidos["trimestre"].replace(tri)


#familia

familia={"DIGITAL ENERGY":"Otros","IND AUTOMATION OPS":"Industrial Automation","POWER SYSTEMS":"Otros",
         "HOME & DISTRIBUTION":"Home & Distribution","POWER PRODUCTS":"Power Products","GLOBAL SERVICES":"Otros","SECURE POWER":"Otros"}
pedidos["Familia"]=pedidos["Familia"].replace(familia)



<ipython-input-2-6df2ecc7e2f8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos["Año"]=pedidos["Año"].str.replace(",","")
<ipython-input-2-6df2ecc7e2f8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos["Año"]=pedidos["Año"].astype("int64")


#Meta trimestral

#Meta anual

In [ ]:
meta_anual=pedidos.groupby(["Familia"],as_index=False)[["Total bruto"]].sum()
meta_anual.head()

,Familia,Total bruto
0,Home & Distribution,417131.15
1,Industrial Automation,878739.82
2,Otros,1286429.92
3,Power Products,833160.07


In [ ]:
meta_tencion=pedidos.groupby(["Familia"],as_index=False)[["Total bruto","Monto total atendido","Monto total no atendido"]].sum()
meta_tencion.head()

,Familia,Total bruto,Monto total atendido,Monto total no atendido
0,Home & Distribution,417131.15,408313.87,8817.28
1,Industrial Automation,878739.82,873261.74,5478.08
2,Otros,1286429.92,1284028.82,2401.10
3,Power Products,833160.07,832233.65,926.42


In [ ]:
meta_tencion["Total bruto"].sum()


3415460.9599999995

In [ ]:
meta_tencion["Monto total atendido"].sum()

3397838.0799999996

In [ ]:
meta_tencion["Monto total no atendido"].sum()

17622.879999999997

In [ ]:
#pedidos.info()
#Meta_2024=pd.Series((pedidos["Total bruto"].sum()),index=[Meta 2024])
#Meta_2024.head()



#Meta por trimetres

In [ ]:
trimestre=pedidos.pivot_table(index="Familia",columns=["Año","trimestre"],values="Total bruto",aggfunc="sum")
trimestre.head()



Año                          2023                                       \
trimestre             Trimestre 2 Trimestre 3 Trimestre 4 Trimestres 1   
Familia                                                                  
Home & Distribution      74533.86    18590.60    25543.11      1602.87   
Industrial Automation   153513.88    47676.88    44061.46     21100.29   
Otros                    40541.66   535841.71    86296.63    533563.23   
Power Products           85116.30    95873.44    90712.47      4984.48   

Año                          2024                                       
trimestre             Trimestre 2 Trimestre 3 Trimestre 4 Trimestres 1  
Familia                                                                 
Home & Distribution      45795.21   108186.63    78889.43     63989.44  
Industrial Automation    38550.64   242977.18    84765.22    246094.27  
Otros                     6295.49    59738.24    12394.71     11758.25  
Power Products           44304.26   222730.89   114554.57    174883.66

#Exportar a excel

In [ ]:
pedidos.to_excel("pedidos.xlsx",index=False,sheet_name="Formato pedido")

trimestre.to_excel("tri.xlsx",sheet_name="Formato pedido")